# 4.8  Grouping & Aggregating 

#This Script Contains following:

#1 Importing Libraries

#2 Importing path & Dataset

#3 Grouping of Data

#4 Performing multiple aggregation

#5 Creating Flag 

#6 Deriving Columns with loc()

#7 Exporting Clean & aggregated Dataset in pickle format

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import os

In [2]:
#Defining path
path=r'C:\Users\User01\16.05.2023 Instacart Basket Analysis'

In [3]:
# Importing order_products_merged
ords_prods_merge= pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_updated.pkl'))

# Grouping data as per Product Name

In [4]:
ords_prods_merge.groupby('product_name')

# Performing multiple aggregation on entire dataset

In [5]:
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

order_number        
                      mean min max
department_id                     
1                15.455255   1  99
2                17.248522   1  99
3                17.169687   1  99
4                17.808925   1  99
5                15.212750   1  99
6                16.442749   1  99
7                17.227292   1  99
8                15.342914   1  99
9                15.904623   1  99
10               20.257219   1  99
11               16.172317   1  99
12               15.936575   1  99
13               16.568776   1  99
14               16.776753   1  99
15               16.211907   1  99
16               17.676900   1  99
17               15.676384   1  99
18               19.299330   1  99
19               17.176997   1  99
20               16.475588   1  99
21               22.906849   1  99

# Results are same as that of subset data since, minimum orders will always be 1 and the maximum has been capped at 99 by the data engineers at Instacart.

# Create a loyalty flag using loc

In [6]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [7]:
ords_prods_merge.head(10)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,196,1,0,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy day,Average orders,10
1,2398795,1,2,3,7,15.0,196,1,1,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Average orders,10
2,473747,1,3,3,12,21.0,196,1,1,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Most orders,10
3,2254736,1,4,4,7,29.0,196,1,1,195,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,Average orders,10
4,431534,1,5,4,15,28.0,196,1,1,195,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,Most orders,10
5,3367565,1,6,2,7,19.0,196,1,1,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy day,Average orders,10
6,550135,1,7,1,9,20.0,196,1,1,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,10
7,3108588,1,8,1,14,14.0,196,2,1,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,10
8,2295261,1,9,1,16,0.0,196,4,1,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,10
9,2550362,1,10,4,8,30.0,196,1,1,195,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,Average orders,10


# Deriving Columns with loc()

In [8]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [9]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [10]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [11]:
ords_prods_merge['loyalty_flag'].value_counts(dropna=False)

Regular customer    15553342
Loyal customer      10093365
New customer         6114570
Name: loyalty_flag, dtype: int64

# Determine mean of product prices by loyalty group

In [12]:
# Group by "prices" 
ords_prods_merge.groupby('prices')

In [13]:
# Split the data into groups by "loyalty_flag" and then calculate the mean for each "prices" column
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean']})

,prices
,mean
loyalty_flag,
Loyal customer,7.641987
New customer,7.645179
Regular customer,7.645539


# Creating Spending Flag of each user based on Avg. Price

In [14]:
# Group data by "user_id" column, generate mean product price for each user, create "mean_price" for aggregation results
ords_prods_merge['mean_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [15]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0,...,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price
0,2539329,1,1,2,8,NaN,196,1,0,195,...,7,9.0,both,Mid-range product,Regularly busy,Regularly busy day,Average orders,10,New customer,6.391379
1,2398795,1,2,3,7,15.0,196,1,1,195,...,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.391379
2,473747,1,3,3,12,21.0,196,1,1,195,...,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.391379
3,2254736,1,4,4,7,29.0,196,1,1,195,...,7,9.0,both,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.391379
4,431534,1,5,4,15,28.0,196,1,1,195,...,7,9.0,both,Mid-range product,Least busy,Least busy days,Most orders,10,New customer,6.391379


# Deriving Spender Column with loc()

In [16]:
ords_prods_merge.loc[ords_prods_merge['mean_price'] >=10, 'spender_flag'] = 'High Spender'

In [17]:
ords_prods_merge.loc[ords_prods_merge['mean_price'] <10, 'spender_flag'] = 'Low Spender'

In [18]:
ords_prods_merge['spender_flag'].value_counts(dropna=False)

Low Spender     31705481
High Spender       55796
Name: spender_flag, dtype: int64

# Create order frequency flag that indicates users' ordering behavior.

In [19]:
# Group data by "user_id" column, generate median product of "days since prior order, create "median_days" for aggregation results
ords_prods_merge['median_days'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [20]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0,...,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price,spender_flag,median_days
0,2539329,1,1,2,8,NaN,196,1,0,195,...,both,Mid-range product,Regularly busy,Regularly busy day,Average orders,10,New customer,6.391379,Low Spender,20.0
1,2398795,1,2,3,7,15.0,196,1,1,195,...,both,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.391379,Low Spender,20.0
2,473747,1,3,3,12,21.0,196,1,1,195,...,both,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.391379,Low Spender,20.0
3,2254736,1,4,4,7,29.0,196,1,1,195,...,both,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.391379,Low Spender,20.0
4,431534,1,5,4,15,28.0,196,1,1,195,...,both,Mid-range product,Least busy,Least busy days,Most orders,10,New customer,6.391379,Low Spender,20.0


In [21]:
# Create flags to assign "freqency" labels to user ID based on median days since prior order
ords_prods_merge.loc[ords_prods_merge['median_days'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [22]:
ords_prods_merge.loc[(ords_prods_merge['median_days'] > 10) & (ords_prods_merge['median_days'] <= 20), 'frequency_flag'] = 'Regular customer'

In [23]:
ords_prods_merge.loc[ords_prods_merge['median_days'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [24]:
# Check frequencies of spending flags for accuracy
ords_prods_merge['frequency_flag'].value_counts(dropna=False)

Frequent customer        21141387
Regular customer          7057103
Non-frequent customer     3562776
NaN                            11
Name: frequency_flag, dtype: int64

In [25]:
# Examining the null values
print(ords_prods_merge[ords_prods_merge.frequency_flag.isnull()])

          order_id  user_id  order_number  orders_day_of_week  \
363409      228286   163549             1                   5   
3621357     228286   163549             1                   5   
11394718    228286   163549             1                   5   
13645692    895835   159838             1                   0   
16410106    228286   163549             1                   5   
17251990    895835   159838             1                   0   
17622767    895835   159838             1                   0   
23217772    824952   169017             1                   5   
24138593    895835   159838             1                   0   
24986390    228286   163549             1                   5   
25880002    895835   159838             1                   0   

          order_hour_of_day  days_since_prior_order  product_id  \
363409                   10                     NaN       13176   
3621357                  10                     NaN       21137   
11394718          

# The null values for 'frequency flag' and 'median days' for New Customers had only 1 order as days_since_prior_order is null, median cannot be calculated.

In [26]:
# Changing the null values in "frequency_flag" column to "Non-frequent customer"
ords_prods_merge['frequency_flag'].fillna("Non-frequent customer", inplace = True)

In [27]:
ords_prods_merge['frequency_flag'].value_counts(dropna=False)

Frequent customer        21141387
Regular customer          7057103
Non-frequent customer     3562787
Name: frequency_flag, dtype: int64

In [28]:
# Finding Outliers
ords_prods_merge.loc[ords_prods_merge['prices'] > 100]

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price,spender_flag,median_days,frequency_flag


In [29]:
# Marking Outliers as missing to prevent skewing of data
ords_prods_merge.loc[ords_prods_merge['prices'] >100, 'prices'] = np.nan

In [32]:
ords_prods_merge['prices'].max()

25.0

In [30]:
# Export data to pkl
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_aggregated.pkl'))